# Using pandas and scikit-learn for classification tasks

## Getting Started


```bash
git clone git@github.com:jseabold/depy.git
cd depy
pip install -r requirements.txt
```

## About Me

Skipper Seabold <br />
Data Scientist, [Civis Analytics](https://civisanalytics.com/)

<link rel="stylesheet" href="//maxcdn.bootstrapcdn.com/font-awesome/4.3.0/css/font-awesome.min.css">
<img src=https://g.twimg.com/twitter-bird-16x16.png style="float:left"></img> &nbsp; <a href=https://twitter.com/jseabold>@jseabold</a> <br />
<i class="fa fa-github fa-lg"></i> &nbsp; <a href="https://github.com/jseabold">jseabold</a>

## Overview

* Hitting the highlights
* Pandas for Data Wrangling
  * Overview
  * Reading data
  * Exploration
  * GroupBy
  * Plotting
  * Advanced Indexing
  * Categorical Data
* Scikit-Learn for Classification
  * API Overview
  * Preprocessing
  * Decision Trees
  * Gradient Boosting Trees
  * Random Forests
  * Cross-Validation
  * Early Stopping
  * Custom Transformers
  * Pipelines

### Common Imports

In [ ]:
import os
import numpy as np
import pandas as pd

## Notebook Specifics

In [ ]:
%matplotlib inline
pd.set_option("max_rows", 10)
np.set_printoptions(suppress=True)

## Pretty Graphs

In [ ]:
from seaborn import set_style
set_style("darkgrid")
import seaborn as sns
import matplotlib.pyplot as plt

## About the Data

In [ ]:
with open("data/adult.names") as fin:
    notes = fin.read()
    
print(notes)

# Pandas for Data Wrangling

## Reading Data

In [ ]:
dta = pd.read_csv("data/adult.data.cleaned.csv.gz", compression="gzip")

In [ ]:
test = pd.read_csv("data/adult.test.cleaned.csv.gz", compression="gzip")

## Explore the Data

In [ ]:
dta.head()

In [ ]:
dta.info()

In [ ]:
dta.describe()

## Pandas Orientation

### Indices

#### Index

In [ ]:
dta.index

#### Columns

In [ ]:
dta.columns

In [ ]:
dta.columns.difference(test.columns)

Sanity checks

In [ ]:
dta.columns.equals(test.columns)

In [ ]:
dta.columns.difference(test.columns)

#### Indexing

In [ ]:
dta.ix[[5, 10, 15]]

#### Selecting Columns

In [ ]:
dta[["workclass", "education"]]

In [ ]:
type(dta[["workclass"]])

In [ ]:
type(dta["workclass"])

#### Rows and Columns

In [ ]:
dta.ix[[5, 10, 15], ["workclass", "education"]]

## GroupBy Operations

In [ ]:
dta.groupby("income").education.describe()

In [ ]:
grouper = dta.groupby("education")

In [ ]:
grouper

In [ ]:
education_map = grouper.education_num.unique()
education_map.sort()

with pd.option_context("max_rows", 20):
    print(education_map)

In [ ]:
grouper.education_num.apply(lambda x : x.unique()[0])
education_map.sort()

with pd.option_context("max_rows", 20):
    print(education_map)

## Plotting

In [ ]:
ax = dta.groupby("education").size().plot(kind="barh", figsize=(8, 8))

# ax.set_xticklabels([])  # turn off x tick labels

# resize y label
ylabel = ax.yaxis.get_label()
ylabel.set_fontsize(24)

# resize x tick labels
labels = ax.yaxis.get_ticklabels()
[label.set_fontsize(20) for label in labels];

# resize y tick labels
labels = ax.xaxis.get_ticklabels()
[label.set_fontsize(20) for label in labels]
[label.set_rotation(-45) for label in labels];

### Seaborn

In [ ]:
import seaborn as sns

In [ ]:
g = sns.factorplot("education_num", "hours_per_week", hue="sex", col="income", data=dta)

### Deleting Columns

In [ ]:
del dta["education"]
del dta["fnlwgt"]
del test["education"]
del test["fnlwgt"]

### Advanced Indexing

#### Indexing with Booleans

In [ ]:
dta.education_num <= 8

In [ ]:
dta.ix[dta.education_num <= 8, "education_num"]

#### .iloc vs .loc

In [ ]:
dta.ix[dta.education_num <= 8, "education_num"].iloc[0]

In [ ]:
dta.ix[dta.education_num <= 8, "education_num"].loc[3]

#### Slicing with labels (!)

In [ ]:
dta.groupby("workclass").age.mean()

In [ ]:
dta.groupby("workclass").age.mean().ix["Federal-gov":"Private"]

#### Filtering Columns with Regex

In [ ]:
dta.filter(regex="capital")

### Working with Categorical Data

#### Categorical Object

In [ ]:
cat = pd.Categorical(dta.workclass)
cat.describe()

In [ ]:
cat

In [ ]:
cat.categories

In [ ]:
cat.codes

#### Vectorized string operations

In [ ]:
dta.workclass.str.contains("\?")

#### Putting it together: Strings and Boolean Indexing

In [ ]:
dta.ix[dta.workclass.str.contains("\?"), "workclass"]

#### Putting it together: Column Assignment

In [ ]:
dta.workclass.unique()

In [ ]:
for col in dta:  # iterate through column names
    # only look at object types
    if not dta[col].dtype.kind == "O":
        continue
    
    # Replace "?" with "Other"
    if dta[col].str.contains("\?").any():
        dta.ix[dta[col].str.contains("\?"), col] = "Other"
        test.ix[test[col].str.contains("\?"), col] = "Other"

In [ ]:
dta.workclass.unique()

#### Replacing values using dictionaries

In [ ]:
dta.income

In [ ]:
dta.income.replace({"<=50K": 0, ">50K": 1})

In-place changes

In [ ]:
dta.income.replace({"<=50K": 0, ">50K": 1}, inplace=True)

In [ ]:
test.income.replace({"<=50K.": 0, ">50K.": 1}, inplace=True)

In [ ]:
dta.income.mean()

In [ ]:
test.income.mean()

# Classification with Scikit-Learn

## Scikit-Learn API

* Base object is the estimator
* Any object that learns from data
  * Classification, regression, clustering, or transformer 
  
* parameters passed to estimator

```python
    estimator = Estimator(*args, **kwargs)
```

* `fit` method provided

```python
    estimator.fit(X, y)
```
    
* Computed parameters have an underscore appended

```python
    estimator.coef_
```

## Preparing the Data

* scikit-learn works with numerical data

In [ ]:
y = dta.pop("income")
y_test = test.pop("income")

In [ ]:
dta.info()

## Preprocessing

* Preprocessing for Text, Categorical variables, Standardization etc.

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
dta.native_country.head(15).values

In [ ]:
binarizer = LabelBinarizer()

`fit_transform` is short hand for calling `fit` then `transform`

In [ ]:
binarizer.fit_transform(dta.native_country.head(15))

In [ ]:
binarizer.classes_

Pre-processing with pandas

In [ ]:
X_train = pd.get_dummies(dta)

In [ ]:
X_test = pd.get_dummies(test)

Deal with real life

In [ ]:
X_train.columns.equals(X_test.columns)

In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train.columns.difference(X_test.columns)

In [ ]:
X_test[X_train.columns.difference(X_test.columns)[0]] = 0

Preserve order

In [ ]:
X_test = X_test[X_train.columns]

## Reported Benchmarks

```
|    Algorithm               Error
| -- ----------------        -----
| 1  C4.5                    15.54
| 2  C4.5-auto               14.46
| 3  C4.5 rules              14.94
| 4  Voted ID3 (0.6)         15.64
| 5  Voted ID3 (0.8)         16.47
| 6  T2                      16.84
| 7  1R                      19.54
| 8  NBTree                  14.10
| 9  CN2                     16.00
| 10 HOODG                   14.82
| 11 FSS Naive Bayes         14.05
| 12 IDTM (Decision table)   14.46
| 13 Naive-Bayes             16.12
| 14 Nearest-neighbor (1)    21.42
| 15 Nearest-neighbor (3)    20.35
| 16 OC1                     15.04
```

## Classification and Regression Trees (CART)

* Partition feature space into a set of rectangles via splits that lead to largest information gain
* Fit simple model in each region (e.g., a constant)
* Captures non-linearities and feature interactions
* Note: not strictly necessary to dummy encode variables

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

In [ ]:
dtree = DecisionTreeClassifier(random_state=0, max_depth=2)

In [ ]:
dtree.fit(X_train, y)

In [ ]:
export_graphviz(dtree, feature_names=X_train.columns)

Run this if you have graphviz installed
    
    !dot -Tpng tree.dot -o tree.png

In [ ]:
from IPython.display import Image
Image("tree.png", unconfined=True)

Fit the full tree and look at the error

In [ ]:
dtree = DecisionTreeClassifier(criterion='entropy', random_state=0)
dtree.fit(X_train, y)

Performs slightly worse than C4.5 with no pruning

In [ ]:
from sklearn import metrics

In [ ]:
metrics.mean_absolute_error(y_test, dtree.predict(X_test))

Beware overfitting!

In [ ]:
dtree = DecisionTreeClassifier(criterion='entropy', random_state=0, max_depth=10)
dtree.fit(X_train, y)
metrics.mean_absolute_error(y_test, dtree.predict(X_test))

## Aside: Saving Models

* All of the scikit-learn models are picklable
* Using joblib directly is often preferable to using pickle

In [ ]:
import joblib

## Ensemble Methods

### Boosting

* Combine many weak classifiers in to one strong one
  * Weak classifier is slightly better than random
* Sequentially apply a classifier to repeatedly modified versions of data
* Each subsequent classifier improves on the mistakes of its predecessor
* For Boosting Trees, the classifier is a decision tree

In [ ]:
# Create a random dataset

import numpy as np
rng = np.random.RandomState(1)
groundX = np.sort(rng.uniform(0, 10, size=250), axis=0)
groundy = np.linspace(1, -1, 250) + np.sin(2*groundX).ravel()
idx = np.random.randint(0, 250, size=30)
idx.sort()
XX = groundX[idx]
yy = groundy[idx]
XX = XX[:, np.newaxis]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree1 = DecisionTreeRegressor(max_depth=2)

tree1.fit(XX, yy)
y1 = tree1.predict(XX)

resid1 = yy - y1
tree1.fit(XX, resid1)

y2 = tree1.predict(XX)
resid2 = y2 - resid1
tree1.fit(XX, resid2)

y3 = tree1.predict(XX)

In [ ]:
fig, ax = plt.subplots(4, 1, figsize=(6, 18), sharey=True, sharex=True)
ax[0].plot(XX, yy, marker='o', ls='', label='observed')
ax[0].plot(groundX, groundy, label='truth')
ax[0].legend(fontsize=16, loc='lower left');
ax[1].plot(XX, yy, marker='o', ls='')
ax[1].plot(XX, y1)
ax[2].plot(XX, resid1, marker='o', ls='')
ax[2].plot(XX, y2)
ax[3].plot(XX, resid2, marker='o', ls='')
ax[3].plot(XX, y3)
fig.suptitle("Residual Fitting", fontsize=24);
fig.tight_layout()

### Gradient Boosting

* Generalizes boosting to any differentiable loss function

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
if not os.path.exists("models/gbt1.pkl"):
    gbt = GradientBoostingClassifier(max_depth=5, n_estimators=1000)
    gbt.fit(X_train, y)
    joblib.dump(gbt, "models/gbt1.pkl")
else:
    gbt = joblib.load("models/gbt1.pkl")

In [ ]:
metrics.mean_absolute_error(y_test, gbt.predict(X_test))

In [ ]:
if not os.path.exists("models/gbt2.pkl"):
    gbt = GradientBoostingClassifier(max_depth=8, n_estimators=1000, subsample=.5, random_state=0,
                                    learning_rate=.001)
    gbt.fit(X_train, y)
    joblib.dump(gbt, "models/gbt2.pkl")
else:
    gbt = joblib.load("models/gbt2.pkl")

In [ ]:
metrics.mean_absolute_error(y_test, gbt.predict(X_test))

### Bagging

* Bootstrap aggregating (bagging)
  * Simple bootstrapping is sampling with replacement
* Fit the same learner to many bootstrap samples and average the results
* Random forests builds on the idea of bagging and uses trees
* Performance similar to boosting but can be easier to train and tune

### Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
if not os.path.exists("models/rf.pkl"):
    rf = RandomForestClassifier(n_estimators=1000, criterion='entropy', n_jobs=4, max_depth=10)
    rf.fit(X_train, y)
    joblib.dump(rf, "models/rf.pkl")
else:
    rf = joblib.load("models/rf.pkl")

In [ ]:
metrics.mean_absolute_error(y_test, rf.predict(X_test))

* Rule of thumb is that you can't really overfit with random forests
  * This is true in general but only to an extent
* Usually ok to grow large forests with full depth trees (problem dependent)
  * Limiting the depth of the trees and the number of trees can be 

In [ ]:
if not os.path.exists("models/rf_full.pkl"):
    rf_full = RandomForestClassifier(n_estimators=1000, criterion='entropy', 
                                     n_jobs=4, max_depth=None)
    rf_full.fit(X_train, y)
    joblib.dump(rf, "models/rf_full.pkl")
else:
    rf_full = joblib.load("models/rf_full.pkl")

In [ ]:
metrics.mean_absolute_error(y_test, rf_full.predict(X_test))

## Validation Methods

### Cross-Validation

* Sampling techniques to ensure low generalization error and avoid overfitting

In [ ]:
from sklearn.cross_validation import StratifiedKFold

cv = StratifiedKFold([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                      0, 0, 0, 0, 0, 0,], n_folds=3)
for idx in cv:
    print("train", idx[0], "test", idx[1])

In [ ]:
from sklearn.grid_search import GridSearchCV

cv = StratifiedKFold(y, n_folds=4)

params = {"max_depth": [3, 5, 7]}
gbt = GradientBoostingClassifier(n_estimators=500, learning_rate=.01)

if not os.path.exists("models/grid_search.pkl"):
    estimator = GridSearchCV(gbt, param_grid=params, verbose=2)
    estimator.fit(X_train, y)
    joblib.dump(estimator, "models/grid_search.pkl")
else:
    estimator = joblib.load("models/grid_search.pkl")

### Out-of-bag estimates and Early-stopping

In [ ]:
gbt = GradientBoostingClassifier(learning_rate=.01, n_estimators=1000, subsample=.5)

In [ ]:
gbt.fit(X_train, y)

In [ ]:
metrics.mean_absolute_error(y_test, gbt.predict(X_test))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.plot(gbt.oob_improvement_)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

Ad-hoc way to do early-stopping

In [ ]:
def monitor(i, self, local_variables):
    start = max(0, i - 4)
    stop = i + 1

    if i > 5 and np.mean(self.oob_improvement_[start:stop]) < 1e-4:
        print("Stopped at {}".format(i))
        return True

In [ ]:
gbt.fit(X_train, y, monitor=monitor)

In [ ]:
print(len(gbt.oob_improvement_))

## Custom Transformers

In [ ]:
def get_obj_cols(dta, index=False):
    """
    dta : pd.DataFrame
    index : bool
        Whether to return column names or the numeric index.
        Default False, returns column names.
    """
    columns = dta.columns.tolist()
    obj_col_names = list(filter(lambda x : dta[x].dtype.kind == "O", 
                                columns))
    if not index:
        return obj_col_names
    else:
        return list(columns.index(col) for col in obj_col_names) 

In [ ]:
obj_cols = get_obj_cols(dta)

for col in obj_cols:
    print(col)

Make a transformer that reliably transforms DataFrames and Arrays

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator


class PandasTransformer(TransformerMixin, BaseEstimator):
    def __init__(self, dataframe):
        self.columns = dataframe.columns
        self.obj_columns = get_obj_cols(dataframe, index=True)
        obj_index = np.zeros(dataframe.shape[1], dtype=bool)
        obj_index[self.obj_columns] = True
        self.obj_index = obj_index
        
        
    def fit(self, X, y=None):
        X = np.asarray(X)
        # create the binarizer transforms
        _transformers = {}
        for col in self.obj_columns:
            _transformers.update({col: LabelBinarizer().fit(X[:, col])})
        
        self._transformers = _transformers
        return self
    
    def transform(self, X, y=None):
        X = np.asarray(X)
        
        dummies = None
        for col in self.obj_columns:
            if dummies is None:
                dummies = self._transformers[col].transform(X[:, col])
            else:
                new_dummy = self._transformers[col].transform(X[:, col])
                dummies = np.column_stack((dummies, new_dummy))
            
        # remove original columns
        X = X[:, ~self.obj_index]
        
        X = np.column_stack((X, dummies))
        
        return X

## Pipelines

* Often it makes sense to do the data transformation, feature extraction, etc. as part of a Pipeline
* Pipelines are flexible and provide the same sklearn API

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
dtree_estimator = Pipeline([('transformer', PandasTransformer(dta)), 
                            ('dtree', dtree)])

In [ ]:
dtree_estimator.fit(dta, y)

In [ ]:
dtree_estimator.named_steps['dtree']

## Questions?